# Working files
- added diagonal to unitlist

In [1]:
rows = 'ABCDEFGHI'
cols = '123456789'

In [2]:
# 
assignments = []

def assign_value(values, box, value):  #? dict, box, value
    """
    Please use this function to update your values dictionary!
    Assigns a value to a given box. If it updates the board record it.
    """

    # Don't waste memory appending actions that don't actually change any values
    if values[box] == value:
        return values

    values[box] = value
    if len(value) == 1:
        assignments.append(values.copy())
    return values


In [3]:
def cross(a, b):
    return [s+t for s in a for t in b]

In [4]:

boxes = cross(rows, cols)
row_units = [cross(r, cols) for r in rows]
column_units = [cross(rows, c) for c in cols]
square_units = [cross(rs, cs) for rs in ('ABC','DEF','GHI') for cs in ('123','456','789')]

#DIAGONALS
diag_topL_bottomR = [[row_units[i][i] for i in range(len(row_units))]]  # Diagonal Top left to bottom right
diag_topR_bottomL = [[row_units[i][len(row_units)-1-i] for i in range(len(row_units))]]  # Diagonal from top right bottom left

In [5]:
# lets add diags to unitlist
#unitlist = row_units + column_units + square_units

unitlist = row_units + column_units + square_units + diag_topL_bottomR + diag_topR_bottomL

In [6]:
units = dict((s, [u for u in unitlist if s in u]) for s in boxes)
peers = dict((s, set(sum(units[s],[]))-set([s])) for s in boxes)

In [7]:
def display(values):
    """
    Display the values as a 2-D grid.
    Input: The sudoku in dictionary form
    Output: None
    """
    width = 1+max(len(values[s]) for s in boxes)
    line = '+'.join(['-'*(width*3)]*3)
    for r in rows:
        print(''.join(values[r+c].center(width)+('|' if c in '36' else '')
                      for c in cols))
        if r in 'CF': print(line)
    return

In [8]:
def clean_values(grid):
    """Convert grid string into {<box>: <value>} dict with '.' value for empties.

    Args:
        grid: Sudoku grid in string form, 81 characters long
    Returns:
        Sudoku grid in dictionary form:
        - keys: Box labels, e.g. 'A1'
        - values: Value in corresponding box, e.g. '8', or '.' if it is empty.
    """
    assert len(grid) == 81, "Input grid must be a string of length 81 (9x9)"
    return dict(zip(boxes, grid))

# WARNING! We've modified this function to return '123456789' instead of '.' for boxes with no value.
# Look at the explanation above in the text.

In [9]:
def grid_values(grid):
    """Convert grid string into {<box>: <value>} dict with '123456789' value for empties.

    Args:
        grid: Sudoku grid in string form, 81 characters long
    Returns:
        Sudoku grid in dictionary form:
        - keys: Box labels, e.g. 'A1'
        - values: Value in corresponding box, e.g. '8', or '123456789' if it is empty.
    """
    values = []
    all_digits = '123456789'
    for c in grid:
        if c == '.':
            values.append(all_digits)
        elif c in all_digits:
            values.append(c)
    assert len(values) == 81
    return dict(zip(boxes, values))

In [10]:
# goes 1st

def eliminate(values):
    """Eliminate values from peers of each box with a single value.

    Go through all the boxes, and whenever there is a box with a single value,
    eliminate this value from the set of values of all its peers.

    Args:
        values: Sudoku in dictionary form.
    Returns:
        Resulting Sudoku in dictionary form after eliminating values.
    """
    solved_values = [box for box in values.keys() if len(values[box]) == 1]
    for box in solved_values:
        digit = values[box]
        for peer in peers[box]:
            values[peer] = values[peer].replace(digit,'')
    return values

In [26]:
# goes 2nd
def only_choice(values):
    """Finalize all values that are the only choice for a unit.

    Go through all the units, and whenever there is a unit with a value
    that only fits in one box, assign the value to this box.

    Input: Sudoku in dictionary form.
    Output: Resulting Sudoku in dictionary form after filling in only choices.
    """
    for unit in unitlist:
        for digit in '123456789':
            dplaces = [box for box in unit if digit in values[box]]
            if len(dplaces) == 1:
                values[dplaces[0]] = digit
    return values

In [ ]:
def naked_twins(values):
    """Eliminate values using the naked twins strategy.
    Args:
        values(dict): a dictionary of the form {'box_name': '123456789', ...}

    Returns:
        the values dictionary with the naked twins eliminated from peers.
    """

    # Find all instances of naked twins
    # Eliminate the naked twins as possibilities for their peers
    twins_solved_values = [box for box in values.keys() if len(values[box]) == 2]
    for box in twins_solved_values:
        print('box {}:value {}'.format(box, values[box]))
        print('box:{}, has peers {}'.format(box, unitlist[box]))

            

            
           

        
    return values


In [12]:
def reduce_puzzle(values):
    """
    Iterate eliminate() and only_choice(). If at some point, there is a box with no available values, return False.
    If the sudoku is solved, return the sudoku.
    If after an iteration of both functions, the sudoku remains the same, return the sudoku.
    Input: A sudoku in dictionary form.
    Output: The resulting sudoku in dictionary form.
    """
    stalled = False

    while not stalled:
        # Check how many boxes have a determined value
        solved_values_before = len([box for box in values.keys() if len(values[box]) == 1])
        
        #NAKED TWIN???
        #values = naked_twins(values)
        # Use the Eliminate Strategy
        values = eliminate(values)
        # Use the Only Choice Strategy
        values = only_choice(values)
        # Check how many boxes have a determined value, to compare
        solved_values_after = len([box for box in values.keys() if len(values[box]) == 1])
        # If no new values were added, stop the loop.
        stalled = solved_values_before == solved_values_after
        # Sanity check, return False if there is a box with zero available values:
        if len([box for box in values.keys() if len(values[box]) == 0]):
            return False
    return values

In [13]:
# goes 3rd
def search(values):
    "Using depth-first search and propagation, try all possible values."
    # First, reduce the puzzle using the previous function
    values = reduce_puzzle(values)
    if values is False:
        return False ## Failed earlier
    if all(len(values[s]) == 1 for s in boxes):
        return values ## Solved!
    # Choose one of the unfilled squares with the fewest possibilities
    n,s = min((len(values[s]), s) for s in boxes if len(values[s]) > 1)
    # Now use recurrence to solve each one of the resulting sudokus, and
    for value in values[s]:
        new_sudoku = values.copy()
        new_sudoku[s] = value
        attempt = search(new_sudoku)
        if attempt:
            return attempt

In [15]:
easy = '..3.2.6..9..3.5..1..18.64....81.29..7.......8..67.82....26.95..8..2.3..9..5.1.3..'
medium = '4.....8.5.3..........7......2.....6.....8.4......1.......6.3.7.5..2.....1.4......'
diag_sudoku_grid = '2.............62....1....7...6..8...3...9...7...6..4...4....8....52.............3'


game_board = grid_values(diag_sudoku_grid)
game_board_clean = clean_values(diag_sudoku_grid)



print('-------------------------------')
print("ORIGINAL BOARD")
display(game_board_clean)

print('-------------------------')
print('SOLVED BOARD')

display(search(game_board))


-------------------------------
ORIGINAL BOARD
2 . . |. . . |. . . 
. . . |. . 6 |2 . . 
. . 1 |. . . |. 7 . 
------+------+------
. . 6 |. . 8 |. . . 
3 . . |. 9 . |. . 7 
. . . |6 . . |4 . . 
------+------+------
. 4 . |. . . |8 . . 
. . 5 |2 . . |. . . 
. . . |. . . |. . 3 
-------------------------
SOLVED BOARD
2 6 7 |9 4 5 |3 8 1 
8 5 3 |7 1 6 |2 4 9 
4 9 1 |8 2 3 |5 7 6 
------+------+------
5 7 6 |4 3 8 |1 9 2 
3 8 4 |1 9 2 |6 5 7 
1 2 9 |6 5 7 |4 3 8 
------+------+------
6 4 2 |3 7 9 |8 1 5 
9 3 5 |2 8 1 |7 6 4 
7 1 8 |5 6 4 |9 2 3 


In [156]:




easy = '..3.2.6..9..3.5..1..18.64....81.29..7.......8..67.82....26.95..8..2.3..9..5.1.3..'
medium = '4.....8.5.3..........7......2.....6.....8.4......1.......6.3.7.5..2.....1.4......'
diag_sudoku_grid = '2.............62....1....7...6..8...3...9...7...6..4...4....8....52.............3'


data = grid_values(diag_sudoku_grid)
clean_data = clean_values(diag_sudoku_grid)
#eminate = eliminate(data)
#ochoice = only_choice(data)

#eliminate_choice = eliminate(data)


#sample = eliminate(data)

In [153]:
nk_dataset = {'A1': '1',
 'A2': '123456789',
 'A3': '4',
 'A4': '123456789',
 'A5': '9',
 'A6': '123456789',
 'A7': '123456789',
 'A8': '6',
 'A9': '8',
 'B1': '9',
 'B2': '5',
 'B3': '6',
 'B4': '123456789',
 'B5': '1',
 'B6': '8',
 'B7': '123456789',
 'B8': '3',
 'B9': '4',
 'C1': '123456789',
 'C2': '123456789',
 'C3': '8',
 'C4': '4',
 'C5': '123456789',
 'C6': '6',
 'C7': '9',
 'C8': '5',
 'C9': '1',
 'D1': '5',
 'D2': '1',
 'D3': '2379',
 'D4': '123456789',
 'D5': '123456789',
 'D6': '123456789',
 'D7': '123456789',
 'D8': '8',
 'D9': '6',
 'E1': '8',
 'E2': '123456789',
 'E3': '379',
 'E4': '6',
 'E5': '123456789',
 'E6': '123456789',
 'E7': '123456789',
 'E8': '1',
 'E9': '2',
 'F1': '6',
 'F2': '4',
 'F3': '23',
 'F4': '123456789',
 'F5': '8',
 'F6': '123456789',
 'F7': '123456789',
 'F8': '9',
 'F9': '7',
 'G1': '7',
 'G2': '8',
 'G3': '1',
 'G4': '9',
 'G5': '2',
 'G6': '3',
 'G7': '6',
 'G8': '4',
 'G9': '5',
 'H1': '4',
 'H2': '9',
 'H3': '5',
 'H4': '123456789',
 'H5': '6',
 'H6': '123456789',
 'H7': '8',
 'H8': '2',
 'H9': '3',
 'I1': '123456789',
 'I2': '6',
 'I3': '23',
 'I4': '8',
 'I5': '5',
 'I6': '4',
 'I7': '1',
 'I8': '7',
 'I9': '9'}

In [154]:
display(nk_dataset)

    1     123456789     4     |123456789     9     123456789 |123456789     6         8     
    9         5         6     |123456789     1         8     |123456789     3         4     
123456789 123456789     8     |    4     123456789     6     |    9         5         1     
------------------------------+------------------------------+------------------------------
    5         1        2379   |123456789 123456789 123456789 |123456789     8         6     
    8     123456789    379    |    6     123456789 123456789 |123456789     1         2     
    6         4         23    |123456789     8     123456789 |123456789     9         7     
------------------------------+------------------------------+------------------------------
    7         8         1     |    9         2         3     |    6         4         5     
    4         9         5     |123456789     6     123456789 |    8         2         3     
123456789     6         23    |    8         5         4     |    1   

In [193]:
units['F3']

[['F1', 'F2', 'F3', 'F4', 'F5', 'F6', 'F7', 'F8', 'F9'],
 ['A3', 'B3', 'C3', 'D3', 'E3', 'F3', 'G3', 'H3', 'I3'],
 ['D1', 'D2', 'D3', 'E1', 'E2', 'E3', 'F1', 'F2', 'F3']]

In [317]:
def peer_twins(values):
    
    twin_values = [box for box in values.keys() if len(values[box]) == 2]
    #print(twin_values)
    for box in twin_values:
        #print(box,values[box])
        digit = values[box]
        #print(box, digit)
        for peer in peers[box]:
            print('box:{}, value;{}, peer:{}, peer value:{}'.format(box, digit,peer, values[peer]))
            #print(box,digit, values[peer],peer)
            if digit == values[peer]:
                print('value matches')
            #values[peer] = values[peer].replace(digit,'')
    #return values
    
def unit_twins(values):
    twin_values = [box for box in values.keys() if len(values[box]) == 2]
    #print(twin_values)
    for box in twin_values:
        print(peers[box])
  
    

In [318]:
unit_twins(nk_dataset)

{'E2', 'B3', 'A3', 'F1', 'F5', 'D2', 'F2', 'C3', 'H3', 'F9', 'I3', 'F8', 'F4', 'F6', 'D1', 'E3', 'G3', 'E1', 'D3', 'F7'}
{'I8', 'B3', 'A3', 'C3', 'I5', 'H3', 'I1', 'G1', 'I4', 'I6', 'F3', 'I2', 'I7', 'G2', 'H2', 'E3', 'G3', 'I9', 'D3', 'H1'}


In [303]:
column_units

[['A1', 'B1', 'C1', 'D1', 'E1', 'F1', 'G1', 'H1', 'I1'],
 ['A2', 'B2', 'C2', 'D2', 'E2', 'F2', 'G2', 'H2', 'I2'],
 ['A3', 'B3', 'C3', 'D3', 'E3', 'F3', 'G3', 'H3', 'I3'],
 ['A4', 'B4', 'C4', 'D4', 'E4', 'F4', 'G4', 'H4', 'I4'],
 ['A5', 'B5', 'C5', 'D5', 'E5', 'F5', 'G5', 'H5', 'I5'],
 ['A6', 'B6', 'C6', 'D6', 'E6', 'F6', 'G6', 'H6', 'I6'],
 ['A7', 'B7', 'C7', 'D7', 'E7', 'F7', 'G7', 'H7', 'I7'],
 ['A8', 'B8', 'C8', 'D8', 'E8', 'F8', 'G8', 'H8', 'I8'],
 ['A9', 'B9', 'C9', 'D9', 'E9', 'F9', 'G9', 'H9', 'I9']]

In [297]:
column_units[0]

['A1', 'B1', 'C1', 'D1', 'E1', 'F1', 'G1', 'H1', 'I1']

In [149]:
twins(eliminate(data))
# B2 57
#C7 35
#F6 57
#H8 46
#I1 17

['B2', 'C7', 'F6', 'H8', 'I1']
B2 57
C7 35
F6 57
H8 46
I1 17


In [126]:
twins(only_choice(eliminate(data)))

['D4', 'G2', 'G3', 'H8']
D4 59
G2 89
G3 29
H8 19


In [101]:
print('CLEAN DATA')
print(display(clean_data))
print('-------------------------------------------------------------------------')
print('EliMINATE')
print(display(eliminate(data)))
print('------------------------------------------------------------------------')
print('ELIMINATE & ONLY CHOICE')
print(display(only_choice(eliminate(data))))
print('------------------------------------------------------------------------')


CLEAN DATA
2 . . |. . . |. . . 
. . . |. . 6 |2 . . 
. . 1 |. . . |. 7 . 
------+------+------
. . 6 |. . 8 |. . . 
3 . . |. 9 . |. . 7 
. . . |6 . . |4 . . 
------+------+------
. 4 . |. . . |8 . . 
. . 5 |2 . . |. . . 
. . . |. . . |. . 3 
None
-------------------------------------------------------------------------
EliMINATE
   2     356789  34789  |1345789  134578  134579 | 13569  1345689   145   
 45789     57    34789  |1345789  134578    6    |   2      1345   14589  
 45689   35689     1    | 34589   23458   23459  |   35      7     45689  
------------------------+------------------------+------------------------
 14579   12579     6    |  457    123457    8    |  1359   12359    1259  
   3      1258    248   |  145      9      1245  |  156    12568     7    
 15789   125789   2789  |   6     12357     57   |   4     123589  12589  
------------------------+------------------------+------------------------
  1679     4      237   | 13579   13567   13579  |   8     12569   12

In [50]:
print('Eliminate')
print(display(eminate))
print('----------------------------------------------------------------')
print('----------------------------------------------------------------')
print('----------------------------------------------------------------')

print('twins after eminate')
print(display(twins(eminate)))

Eliminate
   4      8      3   |   9      2      1   |   6     5789    5   
   9      6     247  |   3      4      5   |   8      8      1   
   25    257     1   |   8      7      6   |   4    23579   2357 
---------------------+---------------------+---------------------
  1345   1345    8   |         3456    2   |   9    134567 34567 
   7      2      9   |   5      3      1   |   1    13456    8   
  1345  13459    6   |   7     3459    8   |   2     1345   345  
---------------------+---------------------+---------------------
  1234  12347        |   6      8      9   |   5    12478   247  
   8      1      47  |   2      5      3   |   17     7      9   
   6      9      5   |   4      1      7   |   3      8      2   
None
----------------------------------------------------------------
----------------------------------------------------------------
----------------------------------------------------------------
twins after eminate
   4      8      3   |   9      2      1   |

In [21]:
print('CLEAN BOARD')
print(display(clean_data))
print('--------------------')
print('BOARD')
print(display(data))

CLEAN BOARD
. . 3 |. 2 . |6 . . 
9 . . |3 . 5 |. . 1 
. . 1 |8 . 6 |4 . . 
------+------+------
. . 8 |1 . 2 |9 . . 
7 . . |. . . |. . 8 
. . 6 |7 . 8 |2 . . 
------+------+------
. . 2 |6 . 9 |5 . . 
8 . . |2 . 3 |. . 9 
. . 5 |. 1 . |3 . . 
None
--------------------
BOARD
123456789 123456789     3     |123456789     2     123456789 |    6     123456789 123456789 
    9     123456789 123456789 |    3     123456789     5     |123456789 123456789     1     
123456789 123456789     1     |    8     123456789     6     |    4     123456789 123456789 
------------------------------+------------------------------+------------------------------
123456789 123456789     8     |    1     123456789     2     |    9     123456789 123456789 
    7     123456789 123456789 |123456789 123456789 123456789 |123456789 123456789     8     
123456789 123456789     6     |    7     123456789     8     |    2     123456789 123456789 
------------------------------+------------------------------+------------